
# 广义线性模型
线性回归模型假设给定特征$x$和参数$\theta$的情况下，标号$y$服从一个高斯分布，即$y|x;\theta\sim\mathcal{N}(\mu,\sigma)$；而逻辑回归假设给定特征$x$和参数$\theta$的情况下，标号$y$服从一个伯努利分布，即$y|x;\theta\sim Bernoulli(\phi)$，这里伯努利分布的参数$\phi$是条件概率$p(y|x;\theta)$，因此每次采样都依赖于$x$；softmax假设给定特征$x$和参数$\Theta$的情况下，标号$y$服从一个多项式分布，即$y|x;\Theta\sim Multinomial(\phi)$，假设有K个类，$\sum_{i=1}^K \phi_i=1$。下文将证明这三个模型都是广义线性模型（GLM）的特例。
## 指数分布族
具有如下形式的分布族称为指数分布族（exponential family）：
$$ p(y;\eta)=b(y)\exp(\eta^T T(y)-a(\eta))$$
其中$\eta$是分布的natural parameter，就是分布待求的参数。$T(y)$是sufficient statistics（大多数情况$T(y)=y$），是关于样本的函数。$a(\eta)$是log normalizer，这是因为$\exp(a(\eta))=\int b(y)\exp(\eta^TT(y))dy$，起到归一化作用。
确定$a,b,T$即确定了一个分布族，而$\eta$是此分布族的参数。当$\eta$变化时，我们得到这个分布族里不同的分布。
指数分布族是个大家族，家族成员有正态分布，指数分布，gamma分布，卡方分布，beta分布，狄利克雷分布，伯努利分布，泊松分布等。
下面我们可以证明高斯分布和伯努利、多项式分布属于指数分布族。
### 高斯分布
高斯分布的pdf为
$$ p(y|\mu,\sigma)=\mathcal{N}(y|\mu,\sigma)=\frac{1}{\sqrt{2\pi}\sigma}\exp(-\frac{(y-\mu)^2}{2\sigma^2})$$
这里我们只考虑$\sigma^2=1$的情况，这样充分统计量就只有1维了，则高斯分布可以简化为
$$ p(y|\mu)=\frac{1}{\sqrt{2\pi}}\exp(-\frac{(y-\mu)^2}{2})=\frac{1}{\sqrt{2\pi}}\exp(-\frac{y^2}{2})\exp(\mu y-\frac{\mu^2}{2})$$
上述高斯分布可以写成指数分布族：
$$\eta = \mu  \\
T(y) = y \\
a(\eta) = \mu^2/2 = \eta^2 /2 \\
b(y)=\frac{1}{\sqrt{2\pi}} \exp(-\frac{1}{2} y^2)$$
以上的推导我做了很大的简化，即认为方差已知，然而做学问不能这么不求甚解，所以还是把方差不知道的情况也推一下呗。
既然方差不知道，那么自然参数就应该是个向量$\eta=(\mu,\sigma^2)^T$。
接下来依然是观察
$$ p(y|\mu,\sigma)=\mathcal{N}(y|\mu,\sigma)=\frac{1}{\sqrt{2\pi}\sigma}\exp(-\frac{(y-\mu)^2}{2\sigma^2})$$
我们可以对$\frac{1}{\sqrt{2\pi}\sigma}$做点手脚，写为
$$\begin{aligned}\frac{1}{\sqrt{2\pi}\sigma}&=\exp(\ln \frac{1}{\sqrt{2\pi}\sigma})=\exp(-\ln \sqrt{2\pi}\sigma)=\exp(\ln\sqrt{\frac{1}{2\pi\sigma^2}})=\exp(\ln\sqrt{\frac{\beta}{2\pi}})\\&=\exp(\frac{1}{2}\ln\ \beta-\frac{1}{2}\ln \ (2\pi))\end{aligned}$$
则原式写为
$$ p(y|\mu,\sigma)=\frac{1}{\sqrt{2\pi}\sigma}\exp(-\frac{(y-\mu)^2}{2\sigma^2})=\exp(\color{blue}{\underbrace{{-\frac{y^2}{2}\beta+\mu y \beta}}_{\eta^TT(y)}}\color{red}{\underbrace{-\frac{\mu^2}{2}\beta+\frac{1}{2}\ln \ \beta}_{a(\eta)}}\color{magenta}{\underbrace{-\frac{1}{2}\ln(2\pi)}_{b(y)}})$$
其中$b(y)=\exp(-\frac{1}{2}\ln(2\pi))$,$\eta=(\mu\beta,-\frac{1}{2}\beta)^T$,$T(y)=(y,y^2)^T$,$a(\eta)=-\frac{\mu^2}{2}\beta+\frac{1}{2}\ln \ \beta$


### 伯努利分布
伯努利分布的概率密度为
$$\begin{aligned}p(y;\phi)&=\phi^y(1-\phi)^{1-y}\\&=\exp(\color{red}{\underbrace{y\log\phi+(1-y)\log(1-\phi)}_{\mbox{opposite of binary crossentropy}}})\\&=\exp(y\log(\frac{\phi}{1-\phi})+\log(1-\phi))\end{aligned}$$
其中$\phi=P(y=1)$
对应的指数分布族参数为：
$$T(y) = y \\ \eta=\log \frac{\phi}{1-\phi}  \\a(\eta) = -\log(1-\phi)=\log(1+e^\eta)  \\ b(y)=1$$
注意到第二行中$exp(\cdot)$里面是二元交叉熵误差（binary crossentropy）的**相反数**，交叉熵衡量了标签y与概率$\phi$的相似度，同时也是logistic regression的loss function。
logistic regression里面，标签y是给定的，而$\phi=\frac{e^\eta}{1+e^\eta}=\frac{1}{1+e^{-\eta}}$是未知的，我们要估计的正是这个概率$\phi$，我们同时还应该注意到$\phi$的形式是一个sigmoid function。

## 广义线性模型
广义线性模型对$p(y|x;\theta)$和模型作了如下三个假设

* 给定$x$和$\theta$，$y$服从参数为$\eta$的指数分布族：$y|x;\theta\sim ExponentialFamily(\eta)$
* 给定$x$，我们的目标是预测$T(y)$的数学期望，通常$T(y)=y$,因此是$h(x)=E[T(y)|x]=E[y|x]$
* 自然参数是$x$的线性函数，即$\eta=\theta^Tx$

利用GLM，我们可以很容易地得出最小二乘、logistic回归，softmax回归。
## 线性回归
期望为
$$h_{\theta}(x)=E[y|x]=\mu=\eta=\theta^Tx$$
可以用最大似然法求解参数$\theta$
## logistic回归
期望为
$$h_{\theta}(x)=E[y|x]=\phi=\frac{1}{1+e^{-\eta}}=\frac{1}{1+e^{-\theta^Tx}}$$
我们依然可以用最大似然法求解参数$\theta$
给定一个训练集$\mathcal{D}=\{(x_1,y_1),...,(x_n,y_n)\}$，假设$y_i$独立同分布，对于每个样本有$p(y_i|x_i,\theta)=\phi^{y_i}(1-\phi)^{1-y_i}$.我们求对数似然：
$$ ln\prod_{i=1}^n p(y_i|x_i,\theta)=\sum_{i=1}^n[y_i\ln(\phi^{(i)})+(1-y_i)\ln(1-\phi^{(i)})]=\sum_{i=1}^n[y_i\ln(\frac{1}{1+\exp(-\theta^Tx_i)})+(1-y_i)\ln(1-\frac{1}{1+\exp(-\theta^Tx_i)})]$$
特别说一下，这里的$\phi$为什么带有上标$(i)$，这是因为每个$y_i$虽然都服从一个伯努利分布，但是其对应的概率不同：$\phi^{(i)}=\frac{1}{1+\exp(-\theta^Tx_i)}$，也就是说$y_i$虽然独立，但**不严格同分布**（形式相同，参数不同）。
最大化对数似然等价于最小化它的相反数，于是我们把目标函数表示为一个交叉熵:
$$J(\theta)=\sum_{i=1}^n-[y_i\ln(\frac{1}{1+\exp(-\theta^Tx_i)})+(1-y_i)\ln(1-\frac{1}{1+\exp(-\theta^Tx_i)})]$$
接下去求导与梯度下降的过程就不推导了。
## softmax
softmax中，$y\in\{1,2,...,k\},(k\geq 2)$，k个类对应有k个参数$\Theta=\{\theta_1,\theta_2,...,\theta_k\}$

定义$T(y)\in\mathbb{R}^{k-1}$，且
$$T(1)=\begin{bmatrix}1\\0\\\vdots\\0\end{bmatrix},T(2)=\begin{bmatrix}0\\1\\\vdots\\0\end{bmatrix},...,T(k-1)=\begin{bmatrix}0\\0\\\vdots\\1\end{bmatrix},T(k)=\begin{bmatrix}0\\0\\\vdots\\0\end{bmatrix}$$
令$\boldsymbol{\phi}=(\phi_1,\phi_2,...,\phi_{k-1})^T$，则$\phi_k=1-\sum_{i=1}^{k-1}\phi_i=1-\boldsymbol{1}^T\boldsymbol{\phi}$，假设$y|x;\Theta\sim Multinomial(\phi^1,...,\phi^k)$（其实这应该是个categorical distribution，但机器学习里通常不区分multinomial和categorical，把两者视作一个东西），那么有
$$\begin{aligned}P(y|x;\theta)&=\phi^{\mathbb{I}(y=1)}_1\phi^{\mathbb{I}(y=2)}_2\cdots \phi^{\mathbb{I}(y=k)}_k\\&=\exp(\sum_{i=1}^{k-1}\mathbb{I}(y=i)\log\phi_i+\mathbb{I}(y=k)\log\phi_k)\\&=\exp((\log\boldsymbol{\phi})^TT(y)+(1-\boldsymbol{1}^TT(y))\log\phi_k)\\&=\exp((\log\boldsymbol{\phi}-\log\phi_k\cdot\boldsymbol{1})^TT(y)+\log\phi_k)\\&=\exp((\log\frac{\boldsymbol{\phi}}{\phi_k})^TT(y)-(-\log\phi_k))\end{aligned}$$
对照指数分布族定义式有：
$$b(y)=1\\\eta=\log\frac{\boldsymbol{\phi}}{\phi_k}\\a(\eta)=-\log\phi_k$$
其中$\eta_i=\log\frac{\phi_i}{\phi_k}$，定义$\eta_k=\log\frac{\phi_k}{\phi_k}=0$
则$e^{\eta_i}=\frac{\phi_i}{\phi_k},(i=1,...,k)$，$\phi_i=\phi_k e^{\eta_i}$，由$\sum_{i=1}^k\phi_i=1$有：
$$\phi_k=\frac{1}{\sum_{j=1}^k e^{\eta_j}}$$
于是$\phi_i=\frac{e^{\eta_i}}{\sum_{j=1}^k e^{\eta_j}}$，代入$\eta_i=\theta_i^Tx$得
$$p(y=i|x;\theta)=\phi_i=\frac{e^{\theta_i^Tx}}{\sum_{j=1}^k e^{\theta_j^Tx}}$$
模型的输出为
$$h_\theta(x)=E\begin{bmatrix}\mathbb{I}(y=1)|x;\theta\\\mathbb{I}(y=2)|x;\theta\\\vdots\\\mathbb{I}(y=k)|x;\theta\end{bmatrix}=\begin{bmatrix}\phi_1\\\phi_2\\\vdots\\\phi_k\end{bmatrix}=\begin{bmatrix}\frac{e^{\theta_1^Tx}}{\sum_{j=1}^k e^{\theta_j^Tx}}\\\frac{e^{\theta_2^Tx}}{\sum_{j=1}^k e^{\theta_j^Tx}}\\\vdots\\\frac{e^{\theta_k^Tx}}{\sum_{j=1}^k e^{\theta_j^Tx}}\end{bmatrix}$$ 

最大似然估计
依然是求对数似然，在进行之前引入一个记号，以表示第i个样本出自第j个类的概率
$$\phi_j^{(i)}=p(y_i=j|x_i,\Theta)=\frac{\exp(\theta_j^Tx_i)}{\sum_{k=1}^K \exp(\theta_k^Tx_i)}$$
$$\begin{aligned} ln\prod_{i=1}^n p(y_i|x_i,\Theta)&=ln \prod_{i=1}^n\phi^{\mathbb{I}(y_i=1)}_1\phi^{\mathbb{I}(y_i=2)}_2\cdots \phi^{\mathbb{I}(y_i=k)}_k=ln\  \phi^{\sum_{i=1}^n \mathbb{I}(y_i=1)}_1\phi^{\sum_{i=1}^n \mathbb{I}(y_i=2)}_2\cdots \phi^{\sum_{i=1}^n \mathbb{I}(y_i=k)}_k \\&=\sum_{i=1}^n[\mathbb{I}(y_i=1)ln\ \phi^{(i)}_1+\mathbb{I}(y_i=2)ln\ \phi^{(i)}_2+...+\mathbb{I}(y_i=k)ln\ \phi^{(i)}_k]\end{aligned}$$
等价于最小化
$$\begin{aligned}J(\Theta)&=-\sum_{i=1}^n \bigg[\mathbb{I}(y_i=1)\ln (\frac{\exp(\theta_1^Tx_i)}{\sum_{k=1}^K \exp(\theta_k^Tx_i)})+...+\mathbb{I}(y_i=1)\ln (\frac{\exp(\theta_K^Tx_i)}{\sum_{k=1}^K \exp(\theta_k^Tx_i)})\bigg]\\&=-\sum_{i=1}^n \sum_{j=1}^K\mathbb{I}(y_i=j) \ln (\frac{\exp(\theta_j^Tx_i)}{\sum_{k=1}^K \exp(\theta_k^Tx_i)})\\&=-\big[\sum_{i=1}^n \sum_{j=1}^K\mathbb{I}(y_i=j)\theta_j^Tx_i -\sum_{i=1}^n \sum_{j=1}^K\mathbb{I}(y_i=j)\ln\sum_{k=1}^K\exp(\theta_k^Tx_i)\big]\\&=-\big[\sum_{i=1}^n \sum_{j=1}^K\mathbb{I}(y_i=j)\theta_j^Tx_i -\sum_{i=1}^n \ln\sum_{k=1}^K\exp(\theta_k^Tx_i)(\sum_{j=1}^K\mathbb{I}(y_i=j))\big]\\&=-\big[\sum_{i=1}^n \sum_{j=1}^K\mathbb{I}(y_i=j)\theta_j^Tx_i -\sum_{i=1}^n \ln\sum_{k=1}^K\exp(\theta_k^Tx_i))\big]\end{aligned}$$

$$\begin{aligned}\frac{\partial J(\Theta)}{\partial \theta_j}&=-\sum_{i=1}^n\frac{\partial\big\{\mathbb{I}(y_i=j)\theta_j^Tx_i-\ln\ \sum_{k=1}^K\exp(\theta_k^Tx_i)\big\}}{\partial \theta_j}\\&=-\sum_{i=1}^n\mathbb{I}(y_i=j)x_i-\frac{\exp(\theta_j^Tx_i)}{\sum_{k=1}^K\exp(\theta_k^Tx_i)}\cdot x_i=-\sum_{i=1}^n(\mathbb{I}(y_i=j)-\phi_j^{(i)})x_i\end{aligned}$$

上式没有解析解，因此使用梯度下降优化。对应的梯度下降更新式为
$$\theta_j := \theta_j-\lambda \frac{\partial J(\Theta)}{\partial \theta_j}=\theta_j+\lambda\sum_{i=1}^n(\mathbb{I}(y_i=j)-\phi_j^{(i)})x_i$$

对于large scale数据，我们通常使用基于mini-batch的梯度下降（mini-batch SGD）来更新参数